## Goal here is to combine Snorkel's weak supervision with labelled data to improve performance of the labels


1.   Use snorkel to create labels for dataset using weak supervision
2.   Fine tune the model with the manually labeleed set.

## Alternative would be to combine snorkel labelling functions and Naive Bayers model to label the data



### Load data


In [1]:
!pip install snorkel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.flush_and_unmount()
from google.colab import drive
drive.mount('/content/drive')
from unicodedata import numeric
from pathlib import Path
from google.colab import drive

import  pandas as pd
import re
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis
from snorkel.labeling.model import LabelModel




Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [3]:
from unicodedata import numeric
from pathlib import Path
from google.colab import drive
import  pandas as pd
import re
import matplotlib.pyplot as plt



BASE_DIR=Path().resolve().parent
GDRIVE_MOUNT=BASE_DIR/"gdrive"
DATA_DIR = GDRIVE_MOUNT / "My Drive/Zennith_Classification/Sample_dataset_labeling"
drive.mount(GDRIVE_MOUNT.as_posix(), force_remount=True)

!ls /gdrive/MyDrive/Zennith_Classification/Sample_dataset_labeling/


labelled = pd.read_csv(DATA_DIR / "Training_set - Sheet1.csv")
labelled.head()

Mounted at /gdrive
'Copy of ZenithBank_Classification.ipynb'   Training_set.gsheet
 Snorkel_label.ipynb			   'Training_set - Sheet1.csv'


,Narration,Category
0,dms purchase reversal_(03/04/24)_213142799996_...,CHARGEBACK
1,pos-(09-11-23)506108**4575-740796-2044svg7-699...,CHARGEBACK
2,two purchase reversal_(02/25/24)_13065120936_2...,CHARGEBACK
3,**rev isw pos bill pyt-712515370360--baxi-----...,CHARGEBACK
4,710611669432 2039cf18 16 03 2024 capricorn dig...,CHARGEBACK


In [4]:
#Create dictionary for mapping 'label' later on
labelled['Category'].nunique()

categories = labelled['Category'].unique()
category_mapping = {i : category for i, category in enumerate(categories)}
category_mapping


{0: 'CHARGEBACK',
 1: 'CORAL PAY SETTLEMENT',
 2: 'CARD SETTLEMENT',
 3: 'WALLET FUNDING',
 4: 'TERM OWNER FEES - POS',
 5: 'VAT RECEIVED',
 6: 'PTSP',
 7: 'BAXI OUTBOUND SETTLEMENT',
 8: 'COMMISSIONS',
 9: 'MOCCA SETTLEMENT',
 10: 'ISW CHARGES',
 11: 'BAXI OUTBOUND OUTFLOW',
 12: 'STAFF SALARY',
 13: 'TRANSPORT',
 14: 'INSURANCE CLAIM INFLOW',
 15: 'POS/MPOS',
 16: 'MICRO LENDING REPAYMENT',
 17: 'CASH IN CASH OUT',
 18: 'CHINA JV SETTLEMENT',
 19: 'FUEL & DIESEL EXPENSE',
 20: 'LOAN INTEREST REPAYMENT',
 21: 'LOAN REPAYMENT',
 22: 'TAX',
 23: 'Bank Charges'}

Define the classes and abstain model


In [5]:
Abstain = -1
Num_classes = 53


## Define weak Labelling functions
 This is rule based labelling that generates boisy signals to label the data

 Whe defining the keywords, ensure to add a " , " at the end to prevent mad ovelapp

In [23]:
@labeling_function()
def lf_0(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "reversal", "2cap/debit side", "mdbo", "rrn/rev/debit side",
        "2039/debit side", "debit side", "rev/debit side", "iswposacqcb",
    )

    # Check if any keyword is in the Narration
    if any(keyword in x.Narration for keyword in keywords):
        return 0
    else:
        return Abstain

@labeling_function()
def lf_1(x):
  return 1 if "4063cdl10000001" in x.Narration else Abstain

@labeling_function()
def lf_2(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "neft/nft", "rrn", "hydrogen", "goodsandservices","2070", "2058",
        "2057", "vat on commision","pos settlement comm","agency cashout",
    )

    # Check if any keyword is in the Narration
    if any(keyword in x.Narration for keyword in keywords):
        return 2
    else:
        return Abstain

# Missing 3 from official sample docs
@labeling_function()
def lf_3(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "wallet",
    )
    if any(keyword in x.Narration for keyword in keywords):
        return 3
    else:
        return Abstain

@labeling_function()
def lf_4(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "terw","term owner",
    )
    if any(keyword in x.Narration for keyword in keywords):
        return 4
    else:
        return Abstain
@labeling_function()
def lf_5(x):
  return 5 if "terwvat" in x.Narration else Abstain

@labeling_function()
def lf_6(x):
  return 6 if "0122588064_4011" in x.Narration else Abstain

@labeling_function()
def lf_7(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "monus/credit side","onafriq/credit side","chameleon/credit side",
    )
    if any(keyword in x.Narration for keyword in keywords):
        return 7
    else:
        return Abstain

## Missing 8

@labeling_function()
def lf_9(x):
  keywords = (
      "paystack",
  )
  if any(keyword in x.Narration for keyword in keywords):
    return 9
  else:
    return Abstain

@labeling_function()

def lf_10(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "acq fee payable","iso cash-out service", "acq_fee_payable",
    )
    if any(keyword in x.Narration for keyword in keywords):
        return 10
    else:
        return Abstain
@labeling_function()

def lf_11(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "monus/debit side","onafriq/debit side",)
    if any(keyword in x.Narration for keyword in keywords):
        return 11
    else:
        return Abstain
@labeling_function()

def lf_12(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "salary","ic contract","24/7 shift",)
    if any(keyword in x.Narration for keyword in keywords):
        return 12
    else:
        return Abstain
@labeling_function()

def lf_13(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "adhoc salary","adhoc salaries",)
    if any(keyword in x.Narration for keyword in keywords):
        return 13
    else:
        return Abstain

@labeling_function()

def lf_14(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "axa mansard",)
    if any(keyword in x.Narration for keyword in keywords):
        return 14
    else:
        return Abstain

# Missing 15

@labeling_function()
def lf_16(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "bridgeway",
    )
    if any(keyword in x.Narration for keyword in keywords):
        return 16
    else:
        return Abstain

@labeling_function()
def lf_17(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "bax_trsf","rsvl/bax_trsf","ssn","mfd","nip sett",
        "dispute sett","rvsl sett",)
    if any(keyword in x.Narration for keyword in keywords):
        return 17
    else:
        return Abstain

@labeling_function()
def lf_18(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "chinajv",)
    if any(keyword in x.Narration for keyword in keywords):
        return 18
    else:
        return Abstain

@labeling_function()
def lf_19(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "web purchase","lang",)
    if any(keyword in x.Narration for keyword in keywords):
        return 19
    else:
        return Abstain

@labeling_function()
def lf_20(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "interest","int",)
    if any(keyword in x.Narration for keyword in keywords):
        return 20
    else:
        return Abstain

@labeling_function()
def lf_21(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "principal payment","msmedf",)
    if any(keyword in x.Narration for keyword in keywords):
        return 21
    else:
        return Abstain

@labeling_function()
def lf_22(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "vat", )
    if any(keyword in x.Narration for keyword in keywords):
        return 22
    else:
        return Abstain

@labeling_function()
def lf_23(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "acct/vat","transactions <1000",)
    if any(keyword in x.Narration for keyword in keywords):
        return 23
    else:
        return Abstain
### How to label transactions <1000

# Missing 24

@labeling_function()
def lf_25(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "9mobile",)
    if any(keyword in x.Narration for keyword in keywords):
        return 25
    else:
        return Abstain

@labeling_function()
def lf_26(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "aba/abba",)
    if any(keyword in x.Narration for keyword in keywords):
        return 26
    else:
        return Abstain

@labeling_function()
def lf_27(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "prepaid wallet",)
    if any(keyword in x.Narration for keyword in keywords):
        return 27
    else:
        return Abstain

@labeling_function()
def lf_28(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "baxi","habaripay","sundry entries","19 digits inflow","30 digits session id",
        "capricorn settlement","pool funding","nip","inflows",)
    if any(keyword in x.Narration for keyword in keywords):
        return 28
    else:
        return Abstain

@labeling_function()
def lf_29(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "bedc",)
    if any(keyword in x.Narration for keyword in keywords):
        return 29
    else:
        return Abstain

@labeling_function()
def lf_30(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "multichoice",)
    if any(keyword in x.Narration for keyword in keywords):
        return 30
    else:
        return Abstain

@labeling_function()
def lf_31(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "ekedc",)
    if any(keyword in x.Narration for keyword in keywords):
        return 31
    else:
        return Abstain

@labeling_function()
def lf_32(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "eedc",)
    if any(keyword in x.Narration for keyword in keywords):
        return 32
    else:
        return Abstain

@labeling_function()
def lf_33(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "glo",)
    if any(keyword in x.Narration for keyword in keywords):
        return 33
    else:
        return Abstain

@labeling_function()
def lf_34(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "ibadan",)
    if any(keyword in x.Narration for keyword in keywords):
        return 34
    else:
        return Abstain

@labeling_function()
def lf_35(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "ie debt","ikeja",)
    if any(keyword in x.Narration for keyword in keywords):
        return 35
    else:
        return Abstain

@labeling_function()
def lf_36(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "jamb",)
    if any(keyword in x.Narration for keyword in keywords):
        return 36
    else:
        return Abstain

@labeling_function()
def lf_37(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "jos",)
    if any(keyword in x.Narration for keyword in keywords):
        return 37
    else:
        return Abstain

@labeling_function()
def lf_38(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "buypower",)
    if any(keyword in x.Narration for keyword in keywords):
        return 38
    else:
        return Abstain

@labeling_function()
def lf_39(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "kano",)
    if any(keyword in x.Narration for keyword in keywords):
        return 39
    else:
        return Abstain

@labeling_function()
def lf_40(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "mpos",)
    if any(keyword in x.Narration for keyword in keywords):
        return 40
    else:
        return Abstain

@labeling_function()
def lf_41(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "tas",)
    if any(keyword in x.Narration for keyword in keywords):
        return 41
    else:
        return Abstain

@labeling_function()
def lf_42(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "mcn",)
    if any(keyword in x.Narration for keyword in keywords):
        return 42
    else:
        return Abstain

@labeling_function()
def lf_43(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "irecharge","greystone","shago","creditswitch","xpress payment",
        "vas2net","remita","palmpay","gotv top up",)
    if any(keyword in x.Narration for keyword in keywords):
        return 43
    else:
        return Abstain

@labeling_function()
def lf_44(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "easypay",)
    if any(keyword in x.Narration for keyword in keywords):
        return 44
    else:
        return Abstain

@labeling_function()
def lf_45(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "pos",)
    if any(keyword in x.Narration for keyword in keywords):
        return 45
    else:
        return Abstain

@labeling_function()
def lf_46(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "nip","trf",)
    if any(keyword in x.Narration for keyword in keywords):
        return 46
    else:
        return Abstain

@labeling_function()
def lf_47(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "smartcash",)
    if any(keyword in x.Narration for keyword in keywords):
        return 47
    else:
        return Abstain

@labeling_function()
def lf_48(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "smile",)
    if any(keyword in x.Narration for keyword in keywords):
        return 48
    else:
        return Abstain

@labeling_function()
def lf_49(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "spectranet",)
    if any(keyword in x.Narration for keyword in keywords):
        return 49
    else:
        return Abstain

@labeling_function()
def lf_50(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "sportbet",)
    if any(keyword in x.Narration for keyword in keywords):
        return 50
    else:
        return Abstain

@labeling_function()
def lf_51(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "startime",)
    if any(keyword in x.Narration for keyword in keywords):
        return 51
    else:
        return Abstain

@labeling_function()
def lf_52(x):
    # Define the list of keywords to check in the Narration
    keywords = (
        "waec",)
    if any(keyword in x.Narration for keyword in keywords):
        return 52
    else:
        return Abstain

## Apply labelling funtions on unlabelled data to create weak labels
Obtain matrix of predictions from labelling functions

In [24]:
# Temporarily removed lf_1 to view performance without it due to rule conflicts
lfs = [lf_0, lf_1, lf_2, lf_4, lf_5, lf_6, lf_7,
       lf_10, lf_11, lf_12, lf_13, lf_14,  lf_16, lf_17, lf_18, lf_19,
       lf_20, lf_21, lf_22, lf_23, ]
"""  lf_25, lf_26, lf_27, lf_28, lf_29,
       lf_30, lf_31, lf_32, lf_33, lf_34, lf_35, lf_36, lf_37, lf_38, lf_39,
       lf_40, lf_41, lf_42, lf_43, lf_44, lf_45, lf_46, lf_47, lf_48, lf_49,
       lf_50, lf_51, lf_52]"""


# Apply labeling functions to dataset
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=labelled)

#L_predictions = applier.apply(df=labelled)
#L_predictions_df = pd.DataFrame(L_predictions)

#print(len(L_predictions_df))
#L_predictions_df



100%|██████████| 185/185 [00:00<00:00, 2851.65it/s]


In [25]:
# 10 min execution time
from snorkel.labeling.model import LabelModel

# Train a LabelModel to combine the labeling functions
label_model = LabelModel(cardinality= Num_classes, verbose=True)
label_model.fit(L_train=L_train, n_epochs=100, log_freq=1000, seed=43, lr=0.001)



100%|██████████| 100/100 [00:08<00:00, 12.28epoch/s]


In [26]:
import numpy as np
# Check how many -1 values are in L_train
print(L_train.shape)
# Check percentage of abstained labels
np.mean(L_train == -1)


(185, 20)


0.9678378378378378

In [27]:
# Get the predicted labels from the LabelModel
labelled['label'] = label_model.predict(L=L_train)
labelled


,Narration,Category,label,Snork_label
0,dms purchase reversal_(03/04/24)_213142799996_...,CHARGEBACK,0,CHARGEBACK
1,pos-(09-11-23)506108**4575-740796-2044svg7-699...,CHARGEBACK,-1,NaN
2,two purchase reversal_(02/25/24)_13065120936_2...,CHARGEBACK,0,CHARGEBACK
3,**rev isw pos bill pyt-712515370360--baxi-----...,CHARGEBACK,19,FUEL & DIESEL EXPENSE
4,710611669432 2039cf18 16 03 2024 capricorn dig...,CHARGEBACK,-1,NaN
...,...,...,...,...
180,mobile transfer to smartcash-fund movement to sma,WALLET FUNDING,-1,NaN
181,pp_capricorn/capricorn aba wallet top up b654r...,WALLET FUNDING,-1,NaN
182,pp_capricorn ibadan wallet top up cv902/pob/fu...,WALLET FUNDING,-1,NaN
183,transactions <1001,Bank Charges,-1,NaN


In [28]:
labelled['label'].nunique()

17

### Map predicted labels which are numeric to their text equivalent.

In [29]:
#labelled['label'] = labelled['label'].map(category_mapping)

labelled['Snork_label'] = labelled['label'].map(category_mapping)
labelled



from google.colab import sheets
sheet = sheets.InteractiveSheet(df=labelled)

https://docs.google.com/spreadsheets/d/1h1g1b-sVK6bSt4CzbUJl3peS0ca6j7aZCMOXVyLMF1U#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


## Model evaluation


1.   Coverage - fraction of datapoints with at least one label
2.   Overlap - fraction of datapoints with at least 2 (non-abstain) labels.
3.   Conflicts - fraction of datapoints each labelling function disagrees with at least one other labelling function.
4.   Polarity - Infer polarities of each LF based on evidence in a label matrix






In [30]:
# Analyze the labeling function outputs
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_0,0,[0],0.048649,0.010811,0.010811
lf_1,1,[1],0.021622,0.000000,0.000000
lf_2,2,[2],0.210811,0.124324,0.124324
lf_4,3,[4],0.032432,0.010811,0.010811
lf_5,4,[5],0.010811,0.010811,0.010811
lf_6,5,[6],0.010811,0.000000,0.000000
lf_7,6,[],0.000000,0.000000,0.000000
lf_10,7,[10],0.010811,0.000000,0.000000
lf_11,8,[],0.000000,0.000000,0.000000
lf_12,9,[12],0.016216,0.000000,0.000000


****** Due to high overlap and conflicts, there is a need to revise the labelling functions, potentially to include bank account number and rules.


## Majority vote
Use of majority vote model to make a class prediction

In [31]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter(cardinality=Num_classes, verbose=True)
prediction = majority_model.predict(L=L_train)

prediction

array([ 0, -1,  0, -1, -1,  0, -1,  0,  1,  1, -1, -1, -1, -1,  2, -1,  2,
       -1, -1,  2,  2, -1, -1, -1, -1,  2,  2, -1, -1, -1, -1, -1, -1, -1,
        4,  4, -1,  6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,
        0,  1,  1, -1, -1, -1, -1, -1, -1, -1,  2, -1, -1, -1, 10, 12, 13,
       12, 12, 14, -1, -1, -1, -1, -1, -1, 16,  2,  4,  4, -1,  6, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1,  2, -1,  2, -1, 17,
       17, 20, -1, -1, -1, 17, -1, -1,  2, -1,  2, 18, -1, 17,  2, 19, 19,
       -1, -1, 17, -1, 20, 20, 21, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, 17, -1, 17, 17, 17, 20,  2, -1, -1,  2, -1, -1, 21, 22, -1,
       -1, -1, -1, 10, 20,  2, 20, -1, -1, -1, -1, -1, -1, -1, -1, -1, 20,
       -1, -1, -1, -1, -1, -1, -1, -1, 22, 20, -1, -1, -1, -1, -1])

## Snorkel only accuracy

In [32]:
# Calculate the number of rows with matching values
matching_rows = labelled[labelled['Category'] == labelled['Snork_label']].shape[0]

# Calculate the total number of rows
total_rows = labelled.shape[0]

# Calculate the percentage of matching rows
matching_percentage = (matching_rows / total_rows) * 100

# Display the results
print(f"Number of rows with matching values: {matching_rows}")
print(f"Total number of rows: {total_rows}")
print(f"Percentage of matching rows: {matching_percentage:.2f}%")

# Display the count of each unique category in 'Category' column
print("\nCount of each unique category in 'Category' column:")
print(labelled['Category'].value_counts())

# Display the count of each unique category in 'Snork_label' column
print("\nCount of each unique category in 'Snork_label' column:")
print(labelled['Snork_label'].value_counts())


Number of rows with matching values: 61
Total number of rows: 185
Percentage of matching rows: 32.97%

Count of each unique category in 'Category' column:
Category
WALLET FUNDING              58
CARD SETTLEMENT             41
CHARGEBACK                  23
CASH IN CASH OUT             9
PTSP                         6
BAXI OUTBOUND SETTLEMENT     6
LOAN INTEREST REPAYMENT      5
TERM OWNER FEES - POS        4
COMMISSIONS                  4
CORAL PAY SETTLEMENT         4
BAXI OUTBOUND OUTFLOW        3
STAFF SALARY                 3
ISW CHARGES                  3
POS/MPOS                     2
FUEL & DIESEL EXPENSE        2
VAT RECEIVED                 2
LOAN REPAYMENT               2
Bank Charges                 2
TRANSPORT                    1
INSURANCE CLAIM INFLOW       1
MICRO LENDING REPAYMENT      1
MOCCA SETTLEMENT             1
CHINA JV SETTLEMENT          1
TAX                          1
Name: count, dtype: int64

Count of each unique category in 'Snork_label' column:
Snork_labe

In [33]:
# prompt: display a table in google sheets with all the differently matched values from the cell above

# Create a DataFrame with the mismatched values
mismatched_df = labelled[labelled['Category'] != labelled['Snork_label']]

# Display the mismatched values in an interactive sheet
sheet = sheets.InteractiveSheet(df=mismatched_df)


https://docs.google.com/spreadsheets/d/12kS6UV3n4oA1ueER4BnaZL_YCUuW56pYpojNwBfcsq8#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


Document overlapping rules:


1.   Chargeback - Fuel and diesel expense
2.   Card settlement - Fuel and diesel expense
3.   Card settlement - Loan interest repayment
4.   Vat received - term owner fees pos
5.   

Inadequate rule definition:



1.   Wallet funding ****
2.   Baxi outbound outflow
3.   Ptsp
4.   pos
5.   commissions
6.   mocha settlement
7.   isw charges

Potential points of improvements:


*   Remove special characters from Narration
*   List item





